<a href='http://www.holoviews.org'><img src="assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Excercise 4: Dynamic Interactions</h2></div>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv

hv.extension('bokeh')

### Exercise 1

In [ ]:
diamonds = pd.read_csv('../data/diamonds.csv')
diamonds.head()

In [ ]:
%%opts Scatter [width=600 height=400 logy=True tools=['lasso_select'] color_index='cut'] (size=1 cmap='tab20c')

scatter = hv.Scatter(diamonds.sample(20000), 'carat', ['price', 'cut', 'clarity']).select(carat=(0, 3))
boxwhisker = hv.BoxWhisker(scatter, 'clarity', 'price')

scatter + boxwhisker

In [ ]:
selection = hv.streams.Selection1D(source=scatter)

def cut_bars(index):
    return hv.BoxWhisker(scatter.iloc[index], 'clarity', 'price')

scatter + hv.DynamicMap(cut_bars, streams=[selection])

## Exercise 2: Streaming Data

In [ ]:
import time
import colorcet
from itertools import cycle
from holoviews.operation.datashader import datashade

from streamz import Stream

def taxi_trips_stream(source='/Users/philippjfr/datashader/examples/data/nyc_taxi.csv', frequency='H'):
    """Generate dataframes grouped by given frequency"""
    def get_group(resampler, key):
        try:
            df = resampler.get_group(key)
            df.reset_index(drop=True)
        except KeyError:
            df = pd.DataFrame()
        return df

    df = pd.read_csv(source,
                     infer_datetime_format=True,
                     usecols=['tpep_pickup_datetime', 'pickup_x', 'pickup_y', 'total_amount'],
                     parse_dates=['tpep_pickup_datetime'])
    df = df.set_index('tpep_pickup_datetime', drop=True)
    df = df.sort_index()
    r = df.resample(frequency)
    chunks = [get_group(r, g) for g in sorted(r.groups)]
    indices = cycle(range(len(chunks)))
    while True:
        yield chunks[next(indices)]

trips = taxi_trips_stream()
init = next(trips)

In [ ]:
def stream_to_ds(data):
    start, end = data[0], data[-1]
    label = '%s - %s' % (str(start.index.min()), str(end.index.max()))
    return hv.Dataset(pd.concat(data).reset_index(), label=label)
    
# Initialize window stream with initial data chunk
pipe = hv.streams.Pipe([init])
dmap = hv.DynamicMap(stream_to_ds, streams=[pipe])

stream = Stream()
stream.sliding_window(10).sink(pipe.send);

In [ ]:
%%opts RGB [width=600 height=600]
points = dmap.map(lambda ds: hv.Points(ds, ['pickup_x', 'pickup_y']), hv.Dataset)
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')
tiles * datashade(points, cmap=colorcet.palette.bgy)

In [ ]:
for i in range(10):
    stream.emit(next(trips))

In [ ]:
%%opts Curve [width=800 height=400] (color='black' line_width=1) {+framewise} Scatter (color='red') Overlay [show_legend=False]
curve_op = lambda ds: hv.Curve(ds, 'tpep_pickup_datetime', 'total_amount')
amount_dmap = dmap.map(curve_op, hv.Dataset)
minutely = resample(amount_dmap, rule='T', function=np.sum)
minutely * rolling_outlier_std(minutely, rolling_window=10)

In [ ]:
for i in range(100):
    time.sleep(0.1)
    stream.emit(next(trips))